# Lấy danh sách báo cáo trên Github


In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys # cho phép ta thao tác với enter key như type st in séarch bar and hit enter
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re



path = "C:\Program Files (x86)\chromedriver.exe"
driver =  webdriver.Chrome(path)
driver.get('https://github.com/QuyNguyenALC/reportgobiz.github.io/blob/main/SUMMARY.md')


main = driver.find_element_by_id("readme")


elems = main.find_elements_by_xpath("//*[@href]")

list_reports = []

for elem in elems:
    if 'datastudio.google.com' in elem.get_attribute("href"):
        list_reports.append(elem.get_attribute("href"))

driver.quit()

list_reports = [(lambda x: re.sub('/edit','',x))(x) for x in list_reports]

# Duyệt báo cáo check lỗi

In [2]:
from selenium.webdriver.common.action_chains import ActionChains
import re
import time


path = "C:\Program Files (x86)\chromedriver.exe"
driver =  webdriver.Chrome(path)

#Mở giao diện đăng nhập gamil
driver.get('https://datastudio.google.com/reporting/a67db283-f073-4055-ad1a-ede80111ff90/page/SLWoB/edit')

# đăng nhập gmail
driver.find_element_by_name("identifier").send_keys("team_bi@gobiz.vn")
# Click next button
driver.find_element_by_id('identifierNext').click()
time.sleep(3)
# Tìm element password field
password_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'password')))

password_enter = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'input')))

password_enter = password_field.find_element_by_tag_name('input')
password_enter.send_keys("Ptdl@2021")
# Click next button
driver.find_element_by_id('passwordNext').click()
time.sleep(3)


# Duyệt báo cáo

for i in list_reports:
    report_error = str(i)
    try:
        driver.get(i)
        try:
            page_infor = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'pageInfo')))    #Lấy thông tin số trang của báo cáo
        except:
            report_error = report_error + 'Trang chỉ có 1 page/sai layout, '
            try:
                error = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'error')))
                report_error = report_error + 'Có lỗi báo cáo ' 
            except:
                report_error = report_error + 'Báo cáo không có vấn đề, '
            
        # lấy số trang từ thông tin có được 
        page_num = int(re.findall(r'\d+ \w+ (\d+)',page_infor.text)[0])

        # Lấy thẻ back page và next page
        try:
            back = WebDriverWait(driver,25).until(
                EC.presence_of_element_located((By.CLASS_NAME,'preBtn')))
            back = driver.find_element_by_class_name("preBtn")
        except:
            report_error = report_error + "Không tìm thấy thẻ back, "
            
        try:
            next = WebDriverWait(driver,25).until(
                EC.presence_of_element_located((By.CLASS_NAME,'nextBtn')))
            next = driver.find_element_by_class_name("nextBtn")
        except:
            report_error = report_error + "Không tìm thấy thẻ next, " 
        
        

        # Back về trang đầu tiên
        for j in range (8):
            back.click()
            
        # Next trang và duyệt xem có lỗi không    
        for k in range(page_num-1):
            next.click()
            try:
                error = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'error')))
                mess ='Có lỗi page ' + str(k+2)
                report_error = report_error + " : "+  mess + " , "
            except:
                None
    except Exception as e:
            loi_khac = ' lỗi phát sinh khác ' +  str(e)
            report_error = report_error + " : "+  loi_khac
    print(report_error)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=91.0.4472.164)
